# Gerekli kütüphanelerin eklenmesi

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# Veri seti hazırlama fonksiyonların tanımlanması

In [2]:
#rastgele verilerin oluşturulması

def age():
    return np.random.randint(18,80)

def gender():
    return np.random.choice(["kadın", "erkek"])

def health():
    return np.random.choice(["iyi", "hasta"])

def job():
    return np.random.choice(["yonetici", "memur", "isci", "issiz"])

def income():
    return np.random.choice(["iyi", "orta", "kotu"])

def salary():
    return (math.ceil(np.random.randint(0,20000))/10)*10

def debt():
    return np.random.choice(["var", "yok"])

def heritage():
    return np.random.choice(["var", "yok"])

def house():
    return np.random.choice(["var", "yok"])

def car():
    return np.random.choice(["var", "yok"])

def credit():
    return np.random.choice(["evet", "hayir"])

columns = ['yas', 'cinsiyet', 'saglik', 'is', 'gelir',
           'maas', 'borc', 'miras', 'ev', 'araba', 'kredi']

#oluşturulan verilerin veri setine dönüştürülmesi

def generate_data(number):
    data = []
    for i in range(0, number):
        row = [age(), gender(), health(), job(), income(),
           salary(), debt(), heritage(), house(), car(), credit()]
        data.append(row)
        
    return data
        
def generate_df(number):
    data = []
    for i in range(0, number):
        row = [age(), gender(), health(), job(), income(),
           salary(), debt(), heritage(), house(), car(), credit()]
        data.append(row)
    return pd.DataFrame(data, columns=columns)

# Veri setinin oluşturulması

In [3]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        
        #karar ağacı için değişkenler
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        #yaprak node'ları için sonuç değeri
        self.value = value

# Decision tree sınıfının oluşturulması

In [4]:
class DecisionTree():
    
    #constructor
    def __init__(self, min_sample_count, max_depth=4):
        
        self.min_sample_count = min_sample_count
        self.max_depth = max_depth
        self.root = None
        
    def select_best_split(self, dataset, sample_count, feature_count):
        
        best_split={}
        best_info_gain = -1
        
        # her bir sütun/attribute değerini tekrarlanması
        for feature_index in range(feature_count):
            feature_values = X[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            
            #bir feature'un tüm olası değerlerinin tekrarlanması
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    temp_info_gain = self.info_gain(y, left_y, right_y, "gini")
                    if temp_info_gain>best_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_data"] = dataset_left
                        best_split["right_data"] = dataset_right
                        best_split["info_gain"] = temp_info_gain
                        best_info_gain = temp_info_gain
        
        return best_split
                
    #veriyi bölme
    def split(self, dataset, feature_index, threshold):
        
        type_of_feature = self.feature_type(dataset, feature_index)
        
        if type_of_feature=="Continuous":
            dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
            dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        else:
            dataset_left = np.array([row for row in dataset if row[feature_index]==threshold])
            dataset_right = np.array([row for row in dataset if row[feature_index]!=threshold])
        
        return dataset_left, dataset_right

    #modeli eğitme
    def fit(self, X, Y):
       
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    #çoklu tahmin yapma
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    # tek bir tahmin yapan fonksiyon
    def make_prediction(self, x, tree):
        
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    #karar ağacının node'larının oluşturulması
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        #bir sınıfa ait örnek sayısı minimum örnek sayısından daha az değilse ve maksimum derinliğe ulaşılmadıysa yeni node'lar oluştur
        if num_samples>=self.min_sample_count and curr_depth<=self.max_depth:
            #geçerli durum için en iyi split'i seç
            best_split = self.select_best_split(dataset, num_samples, num_features)
            #information gain'e göre splitin gerçekleştirilmesi
            if best_split["info_gain"]>0:
                # sol kırılım
                left_subtree = self.build_tree(best_split["left_data"], curr_depth+1)
                # sağ kırılım
                right_subtree = self.build_tree(best_split["right_data"], curr_depth+1)
                # alt ağaçlarla birlikte bir node ve düğümlerinin oluşturulması
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # yaprak node'lara sonuç değerinin eklenmesi
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    ###################################
    
    #feature tipini belirleme
    def feature_type(self, dataset, index):
        try:
            float(dataset[0][index])
            return "Continuous"
        except ValueError:
            return "Categorical"
    
    #entropi hesabı
    def entropy(values):
        ent = 0.0
        uniques = list(set(values))
        for i in uniques:
            count=0
            for j in values:
                if j==i:
                    count+=1
            probability = count/len(values)
            ent += -probability * np.log2(probability)
        return ent
    
    #gini index hesabı
    
    def gini(self, values):
        unique_values = list(set(values))
        gini = 1
        for value in unique_values:
            gini -= (np.sum(values == value) / len(values)) ** 2
        return gini
    
    
    # entropi ya da gini index kullanarak information gain hesabı
    def info_gain(self, parent, left, right, measure_of_impurity):
    
        left_weight = len(left) / len(parent)
        right_weight = len(right) / len(parent)
        
        if measure_of_impurity=="entropy":
            gain = self.entropy(parent) - left_weight*self.entropy<(left) - right_weight*self.entropy(right)
        else:
            gain = self.gini(parent) - (left_weight*self.gini(left) + right_weight*self.gini(right))
        return gain


In [55]:
# rastgele veri üretimi
row_count = 1000
df = generate_df(row_count)
df

,yas,cinsiyet,saglik,is,gelir,maas,borc,miras,ev,araba,kredi
0,45,kadın,iyi,memur,orta,3014.0,yok,yok,var,var,evet
1,38,kadın,hasta,issiz,kotu,14375.0,yok,var,var,var,evet
2,60,erkek,hasta,isci,iyi,9905.0,var,var,var,var,hayir
3,51,erkek,iyi,yonetici,kotu,14905.0,var,var,var,yok,hayir
4,32,kadın,hasta,isci,kotu,1739.0,yok,yok,yok,yok,evet
...,...,...,...,...,...,...,...,...,...,...,...
995,72,erkek,hasta,isci,iyi,16381.0,var,yok,yok,yok,hayir
996,29,erkek,iyi,isci,orta,4294.0,var,yok,var,var,hayir
997,64,erkek,iyi,issiz,iyi,6903.0,var,yok,yok,var,evet
998,63,erkek,hasta,memur,orta,19406.0,yok,var,yok,var,hayir


In [56]:
#eklenmek istenen veriler

baslik = input("Eklenmek istenen verinin başlığı:")
i=0
secenekler = []
while True:
    temp = input(baslik + " icin " + str(i+1) + ".secenek:")
    if temp == "0":
      break
    else:
      secenekler.append(temp)
    i+=1
ek_veri=[]
for i in range(row_count):
    ek_veri.append(np.random.choice(secenekler))



Eklenmek istenen verinin başlığı:yeni_ozellik_1
yeni_ozellik_1 icin 1.secenek:yatırımcı
yeni_ozellik_1 icin 2.secenek:dilenci
yeni_ozellik_1 icin 3.secenek:hırsız
yeni_ozellik_1 icin 4.secenek:gariban
yeni_ozellik_1 icin 5.secenek:talha
yeni_ozellik_1 icin 6.secenek:0


In [57]:
ek_veri = pd.Series(ek_veri)
df.insert(loc=1, column=baslik, value=ek_veri)

In [58]:
df.shape

(1000, 12)

In [59]:
df

,yas,yeni_ozellik_1,cinsiyet,saglik,is,gelir,maas,borc,miras,ev,araba,kredi
0,45,hırsız,kadın,iyi,memur,orta,3014.0,yok,yok,var,var,evet
1,38,dilenci,kadın,hasta,issiz,kotu,14375.0,yok,var,var,var,evet
2,60,gariban,erkek,hasta,isci,iyi,9905.0,var,var,var,var,hayir
3,51,yatırımcı,erkek,iyi,yonetici,kotu,14905.0,var,var,var,yok,hayir
4,32,hırsız,kadın,hasta,isci,kotu,1739.0,yok,yok,yok,yok,evet
...,...,...,...,...,...,...,...,...,...,...,...,...
995,72,hırsız,erkek,hasta,isci,iyi,16381.0,var,yok,yok,yok,hayir
996,29,gariban,erkek,iyi,isci,orta,4294.0,var,yok,var,var,hayir
997,64,talha,erkek,iyi,issiz,iyi,6903.0,var,yok,yok,var,evet
998,63,dilenci,erkek,hasta,memur,orta,19406.0,yok,var,yok,var,hayir


In [61]:
#veri ön hazırlığı
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=15)

#modelin eğitimi
classifier = DecisionTree(min_sample_count=5, max_depth=6)
classifier.fit(X_train, Y_train)

#accuracy değerinin hesaplanması
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.484

In [36]:
from graphviz import Digraph

basliklar = list(df.columns[:-1])
type(basliklar)
def visualize(node, dot=None):
    if dot is None:
        dot = Digraph()
    if str(node.value)=="None":
        if type(node.threshold)==int or type(node.threshold)==float:
            deger = basliklar[node.feature_index] + "<" + str(node.threshold) + "\n" + "information gain:" + "{:.4f}".format(node.info_gain)
        else:
            deger = basliklar[node.feature_index] + "=" + str(node.threshold) + "\n" + "information gain:" + "{:.4f}".format(node.info_gain)
        
    else:
        if node.value=="evet":
            deger = "kredi alabilir"
        else:
            deger = "kredi alamaz"
    dot.node(str(id(node)), deger, shape='rectangle')
    if node.left:
        dot.edge(str(id(node)), str(id(node.left)), label="evet")
        visualize(node.left, dot)
    if node.right:
        dot.edge(str(id(node)), str(id(node.right)), label="hayır")
        visualize(node.right, dot)
    return dot

# Example usage
node = classifier.root
dot = visualize(node)
dot.render('binary_tree.gv', view=True)


'binary_tree.gv.pdf'

In [37]:
df

,yas,cinsiyet,saglik,is,gelir,maas,borc,miras,ev,araba,kredi,baslik1
0,21,erkek,iyi,isci,orta,15427.0,var,yok,yok,var,hayir,aslik4
1,72,erkek,hasta,issiz,orta,2061.0,var,var,yok,var,hayir,baslik 4
2,76,kadın,hasta,yonetici,kotu,8815.0,yok,yok,yok,var,hayir,aslik4
3,28,erkek,hasta,issiz,iyi,12990.0,var,yok,yok,yok,hayir,aslik4
4,58,erkek,iyi,issiz,kotu,7009.0,yok,yok,var,var,hayir,baslik 4
...,...,...,...,...,...,...,...,...,...,...,...,...
95,39,kadın,hasta,issiz,orta,17718.0,var,var,yok,var,hayir,baslik2
96,66,erkek,hasta,issiz,orta,6355.0,var,yok,var,yok,hayir,baslik2
97,41,erkek,iyi,isci,iyi,3418.0,var,yok,var,var,evet,baslik 4
98,22,erkek,iyi,issiz,kotu,13127.0,yok,yok,yok,var,evet,aslik4


In [32]:
df

,yas,cinsiyet,saglik,is,gelir,maas,borc,miras,ev,araba,kredi
0,21,erkek,iyi,isci,orta,15427.0,var,yok,yok,var,hayir
1,72,erkek,hasta,issiz,orta,2061.0,var,var,yok,var,hayir
2,76,kadın,hasta,yonetici,kotu,8815.0,yok,yok,yok,var,hayir
3,28,erkek,hasta,issiz,iyi,12990.0,var,yok,yok,yok,hayir
4,58,erkek,iyi,issiz,kotu,7009.0,yok,yok,var,var,hayir
...,...,...,...,...,...,...,...,...,...,...,...
95,39,kadın,hasta,issiz,orta,17718.0,var,var,yok,var,hayir
96,66,erkek,hasta,issiz,orta,6355.0,var,yok,var,yok,hayir
97,41,erkek,iyi,isci,iyi,3418.0,var,yok,var,var,evet
98,22,erkek,iyi,issiz,kotu,13127.0,yok,yok,yok,var,evet
